In [4]:
import pandas as pd
import numpy as np
import healpy as hp

from astropy.cosmology import Planck15
from ligo.skymap.io import fits

import os
from os import listdir
from os.path import isfile, join

import sys

H0GLOB=67.9 #69
Om0GLOB=0.3
Xi0Glob =1.
cosmoglob = Planck15
clight = 2.99792458* 10**5#km/s


In [ ]:
def get_ename(fname, verbose=True):

        if len(fname.split('/')[-1].split('_')) <= 2:    #Same as before since certain names contain a "_"
            event_name = fname.split('/')[-1].split('_')[0]
        elif len(fname.split('/')[-1].split('_')) > 2:
            event_name = fname.split('/')[-1].split('_')[0]+'_'+fname.split('/')[-1].split('_')[1]
        else:
            raise ValueError('Could not set event name. Got fname= %s'%fname)
        if verbose:
            print('-- %s' %event_name)
        return event_name
    
def find_ra_dec(nside, pix):
        '''
        input:  pixel ra dec in degrees
        output: (ra, dec) of pixel center in degrees, with nside given by that of the skymap 
        '''
        theta, phi =find_theta_phi(nside,pix)
        ra, dec = ra_dec_from_th_phi(theta, phi)
        return ra, dec    
    
def ra_dec_from_th_phi(theta, phi):
        ra = np.rad2deg(phi)
        dec = np.rad2deg(0.5 * np.pi - theta)
        return ra, dec
    
def find_theta_phi(nside, pix):
        '''
        input:  pixel
        output: (theta, phi)of pixel center in rad, with nside given by that of the skymap 
        '''
        return hp.pix2ang(nside, pix, nest=False)
    
def _read_O3( fname, convert_nested=True):
        
        skymap, metadata = fits.read_sky_map(fname, nest=None, distances=True) #Read the skymap
        event_name = get_ename(fname, verbose=True)
        print('Event: %s' %event_name)
        if (convert_nested) & (metadata['nest']): #If one wants RING ordering (the one of O2 data afaik) just has to set "convert_nested" to True
            p_posterior = hp.reorder(skymap[0],n2r=True)
            mu = hp.reorder(skymap[1],n2r=True)
            sigma = hp.reorder(skymap[2],n2r=True)
            posteriorNorm = hp.reorder(skymap[3],n2r=True)
        else:
            p_posterior= skymap[0]
            mu= skymap[1]
            sigma = skymap[2]
            posteriorNorm= skymap[3]        
        npix = len(p_posterior)
        nside=hp.npix2nside(npix)
        theta,phi=find_theta_phi(nside,np.argmax(p_posterior))
        print('Theta={}, Phi={}'.format(theta, phi))
        ra, dec=find_ra_dec(nside,np.argmax(p_posterior))
        print('Right Ascension={}, Declination={}'.format(ra, dec))
        #print(metadata)
        dl=metadata['distmean']
        std=metadata['diststd']
        z=cosmoglob.H(0).value*dl/clight
        sigz=cosmoglob.H(0).value*std/clight
        print('dL={}Mpc, std={}Mpc'.format(dl,std))
        print('z={}, delta_z={}'.format(z,sigz))
        print('\n')
        head = None

In [3]:
#GWdata path
GW_data_path='/home/rciancarella/DarkSirensStat/data/GW/O3/'
os.chdir(GW_data_path)
#os.listdir()

['GW190814_C01:IMRPhenomPv3HM.fits',
 'GW190706_222641_C01:SEOBNRv4P_nonevol.fits',
 'GW190719_215514_PrecessingSpinIMRHM.fits',
 'GW190426_152155_C01:IMRPhenomPv2.fits',
 'GW190602_175927_C01:SEOBNRv4P.fits',
 'GW190828_065509_PrecessingSpinIMRHM.fits',
 'GW190727_060333_C01:IMRPhenomD.fits',
 'GW190814_combined.fits',
 'GW190630_185205_C01:SEOBNRv4PHM.fits',
 'GW190513_205428_ZeroSpinIMR.fits',
 'GW190727_060333_PrecessingSpinIMR.fits',
 'GW190413_052954_C01:SEOBNRv4PHM.fits',
 'GW190424_180648_C01:SEOBNRv4PHM.fits',
 'GW190803_022701_PrecessingSpinIMRHM.fits',
 'GW190426_152155_AlignedSpinIMRTidal_NSBH.fits',
 'GW190517_055101_C01:IMRPhenomD.fits',
 'GW190930_133541_C01:IMRPhenomPv2.fits',
 'GW190503_185404_C01:IMRPhenomPv2.fits',
 'GW190413_134308_C01:IMRPhenomD.fits',
 'GW190915_235702_PrecessingSpinIMR.fits',
 'GW190519_153544_ZeroSpinIMR.fits',
 'GW190719_215514_PrecessingSpinIMR.fits',
 'GW190828_065509_C01:IMRPhenomPv2.fits',
 'GW190924_021846_C01:SEOBNRv4P.fits',
 'GW190413_0

  # Function to exploit, find how to localise an event and produce a proxy one. 
   def _read_O3(self, fname, convert_nested=True):
        
   skymap, metadata = fits.read_sky_map(fname, nest=None, distances=True) #Read the skymap
        self.event_name = get_ename(fname, verbose=self.verbose)
        if self.verbose:
                print('\nEvent: %s' %self.event_name)
                #Raul: prints for control
                print('Delta=%s. If delta=1, normal run '%delta)
        if (convert_nested) & (metadata['nest']): #If one wants RING ordering (the one of O2 data afaik) just has to set "convert_nested" to True
            self.p_posterior = hp.reorder(skymap[0],n2r=True)
            self.mu = hp.reorder(skymap[1],n2r=True)
            self.sigma = hp.reorder(skymap[2],n2r=True)*delta
            self.posteriorNorm = hp.reorder(skymap[3],n2r=True)
        else:
            self.p_posterior= skymap[0]
            self.mu= skymap[1]
            self.sigma = skymap[2]*delta
            self.posteriorNorm= skymap[3]    

In [ ]:
#Some files to look at 
filename=['GW190412_PublicationSamples.fits','GW190814_PublicationSamples.fits',
          'GW190924_021846_PublicationSamples.fits','GW190527_092055_PublicationSamples.fits',
         'GW190421_213856_PublicationSamples.fits','GW190708_232457_PublicationSamples.fits',
         'GW190915_235702_PublicationSamples.fits']
########################## THIS GIVE US SOME INFO ######################################
#for name in filename:
#    _read_O3(name)
########################################################################################
fname =
skymap, metadata = fits.read_sky_map(fname, nest=None, distances=True)